In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

In [54]:
target_file_dir = "../datas/"
target_file_name = "real_estate_data_20211_20224_14_extract_by_cate"
target_file_end = ".csv"
df = pd.read_csv(target_file_dir + target_file_name + target_file_end)
df['TradePrice'] = df.TradePrice/10000

In [55]:
main_cols = [
    'Type','Prefecture', 'Municipality', 'DistrictName',
    'Structure', 'FloorPlan',
    'TradePrice', 'AgeAtTrade','Area',
    'TotalFloorArea','CoverageRatio', 'FloorAreaRatio',
    'BuildingYearW','TradeYear', 'TradeQuarter'
    ]

In [56]:
df2 = df.sort_values(['Prefecture','MunicipalityCode', 'DistrictName',
                     'BuildingYearW','TradePrice','Area']).copy()
df2['TradeNo'] = range(len(df2))
df2_main = df2.loc[:,['TradeNo']+main_cols]
df2_sub =  df2.drop(main_cols, axis=1)

In [57]:
print(df2.shape, df2_main.shape, df2_sub.shape)

(29548, 33) (29548, 16) (29548, 18)


In [58]:
df2.FloorPlan.unique()

array(['３ＬＤＫ', nan, '４ＬＤＫ', '２ＬＤＫ', '３ＬＤＫ＋Ｓ', '１ＬＤＫ', '１Ｋ', '２ＬＤＫ＋Ｓ',
       '２ＤＫ', '３ＤＫ', '１Ｒ', '１ＤＫ', '４ＤＫ', '２Ｋ', 'スタジオ', 'オープンフロア', '５ＬＤＫ',
       '２ＤＫ＋Ｓ', '１ＬＤＫ＋Ｓ', '３Ｋ', '４ＬＤＫ＋Ｓ', '２ＬＤ＋Ｓ', '３ＬＫ', '４Ｋ', '５ＤＫ'],
      dtype=object)

In [52]:
df2.query('TradeYear==2022').pivot_table(values='Type', index='FloorPlan', aggfunc='count').sort_values('Type')

,Type
FloorPlan,
5LDK,10
others,13
1R,87
4LDK,400
1LDK,495
2LDK,1400
1K,1426
3LDK,3100


In [70]:
import pandas as pd
import numpy as np
import mojimoji

def replace_floor_plan(df):
    df_ = df.copy()
    # 0. 元のFloorPlan列をFloorPlanOrigin列として残す
    df_['FloorPlanOrigin'] = df_['FloorPlan']

    # 1. "FloorPlanFlagDK(K)"列と"FloorPlanFlagS"列と"FloorPlanNumber"列を作る(デフォルト値は0)
    df_['FloorPlanNumber'] = 0

    # 2. 全角のアルファベットと数字と半角にする
    # 全角に変換する関数
    def to_full_width(text):
        return mojimoji.zen_to_han(text)
    # Nanがあるとダメなので置換
    df_['FloorPlan'] = df_.loc[:,['FloorPlan']].fillna('empty')
    # DataFrameの要素を全角に変換
    df_['FloorPlan'] = df_.loc[:,['FloorPlan']].applymap(to_full_width)

    # 3. "FloorPlanSTR"列を作成する。デフォルト値はFloorPlan列とする。その後、FloorPlanSTR列の先頭の文字が数字の場合、その数字を除去する
    df_['FloorPlanSTR'] = df_['FloorPlan'].astype(str)
    df_['FloorPlanSTR'] = df_['FloorPlan'].astype(str).apply(lambda x: x[1:] if x[0].isdigit() else x)

    # 4. FloorPlan列の先頭の文字が数字の行について、その数字でFloorPlanNumber列を更新する
    df_['FloorPlanNumber'] = np.where(df_['FloorPlan'].astype(str).apply(lambda x: x[0].isdigit()),
                                    df_['FloorPlan'].astype(str).apply(lambda x: x[0]), df_['FloorPlanNumber'])

    # 5. FloorPlan列の先頭の文字が数字の行について、その数字を除いた値でFloorPlanSTR列を更新する
    df_['FloorPlanSTR'] = np.where(df_['FloorPlan'].astype(str).apply(lambda x: x[0].isdigit()),
                                df_['FloorPlan'].astype(str).apply(lambda x: x[1:]), df_['FloorPlanSTR'])
    
    # 6. FloorPlanSTR列が"K"か”DK”かつ、FloorPlanNumber列が1でない行について、FloorPlanNumberを-1して、FloorPlanSTRを"LDK"にする
    df_['FloorPlanNumber'] = np.where(((df_['FloorPlanSTR'] == 'K') | (df_['FloorPlanSTR'] == 'DK')) &
                                      (df_['FloorPlanNumber'] != '1'),
                                      df_['FloorPlanNumber'].astype(int) - 1, df_['FloorPlanNumber'])
    df_['FloorPlanSTR'] = np.where(((df_['FloorPlanSTR'] == 'K') | (df_['FloorPlanSTR'] == 'DK')) &
                                   (df_['FloorPlanNumber'] != '1'), 'LDK', df_['FloorPlanSTR'])

    # 7. FloorPlanSTR列が"DK"かつ、FloorPlanNumber列が1の行について、FloorPlanSTRを”K”にする
    df_['FloorPlanSTR'] = np.where((df_['FloorPlanSTR'] == 'DK') & (df_['FloorPlanNumber'] == '1'), 'K', df_['FloorPlanSTR'])

    # 8. FloorPlanSTR列が"LDK+S"の行について、FloorPlanNumberを+1して、FloorPlanSTRを"LDK"にする
    df_['FloorPlanNumber'] = np.where((df_['FloorPlanSTR'] == 'LDK+S'),
                                      df_['FloorPlanNumber'].astype(int) + 1, df_['FloorPlanNumber'])
    df_['FloorPlanSTR'] = np.where((df_['FloorPlanSTR'] == 'LDK+S'), 'LDK', df_['FloorPlanSTR'])

    # 9. FloorPlanSTR列が"DK+S"もしくは"DK+S"の行について、FloorPlanSTRを"LDK"にする
    df_['FloorPlanSTR'] = np.where((df_['FloorPlanSTR'] == 'DK+S') | (df_['FloorPlanSTR'] == 'K+S'), 'LDK', df_['FloorPlanSTR'])

    # 10. FloorPlan列を、FloorPlanNumber列とFloorPlanSTR列を連結した値で更新する
    df_['FloorPlan'] = df_['FloorPlanNumber'].astype(str) + df_['FloorPlanSTR']
    
    # 11. FloorPlanSTRがLDK、もしくはFloorPlanが1Kか1Rのもの以外は、FloorPlanを"others"に
    df_['FloorPlan'] = np.where(~((df_['FloorPlanSTR'] == 'LDK') | (df_['FloorPlan'].isin(['1K', '1R']))),
                            'others', df_['FloorPlan'])
    
    # 12. 元がNanのものはNanに戻す
    df_['FloorPlan'] = np.where(df_['FloorPlanOrigin'].isna(), np.nan, df_['FloorPlan'])
    
    # 13. 不要な列を削除
    df_ = df_.drop(['FloorPlanSTR', 'FloorPlanNumber'], axis=1)

    return df_

# 関数を呼び出して結果を得る
df_result = replace_floor_plan(df_tmp)

# サンプルのDataFrameを作成
data = {'FloorPlan': ['３ＬＤＫ', np.nan, '４ＬＤＫ', '２ＬＤＫ', '３ＬＤＫ＋Ｓ', '１ＬＤＫ', '１Ｋ', '２ＬＤＫ＋Ｓ',
                      '２ＤＫ', '３ＤＫ', '１Ｒ', '１ＤＫ', '４ＤＫ', '２Ｋ', 'スタジオ', 'オープンフロア', '５ＬＤＫ',
                      '２ＤＫ＋Ｓ', '１ＬＤＫ＋Ｓ', '３Ｋ', '４ＬＤＫ＋Ｓ', '２ＬＤ＋Ｓ', '３ＬＫ', '４Ｋ', '５ＤＫ']}

df_tmp = pd.DataFrame(data)

# 結果を表示
result_df = replace_floor_plan(df_tmp)
display(result_df)


<ipython-input-70-d32300f2ed9f>:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_['FloorPlan'] = df_.loc[:,['FloorPlan']].applymap(to_full_width)
<ipython-input-70-d32300f2ed9f>:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_['FloorPlan'] = df_.loc[:,['FloorPlan']].applymap(to_full_width)


,FloorPlan,FloorPlanOrigin
0,3LDK,３ＬＤＫ
1,NaN,NaN
2,4LDK,４ＬＤＫ
3,2LDK,２ＬＤＫ
4,4LDK,３ＬＤＫ＋Ｓ
5,1LDK,１ＬＤＫ
6,1K,１Ｋ
7,3LDK,２ＬＤＫ＋Ｓ
8,1LDK,２ＤＫ
9,2LDK,３ＤＫ


In [71]:
df2.FloorPlan.unique()

array(['３ＬＤＫ', nan, '４ＬＤＫ', '２ＬＤＫ', '３ＬＤＫ＋Ｓ', '１ＬＤＫ', '１Ｋ', '２ＬＤＫ＋Ｓ',
       '２ＤＫ', '３ＤＫ', '１Ｒ', '１ＤＫ', '４ＤＫ', '２Ｋ', 'スタジオ', 'オープンフロア', '５ＬＤＫ',
       '２ＤＫ＋Ｓ', '１ＬＤＫ＋Ｓ', '３Ｋ', '４ＬＤＫ＋Ｓ', '２ＬＤ＋Ｓ', '３ＬＫ', '４Ｋ', '５ＤＫ'],
      dtype=object)

In [72]:
result_df = replace_floor_plan(df2)
result_df.pivot_table(values='Type', index='FloorPlan', aggfunc='count',fill_value=0).sort_values('Type')

<ipython-input-70-d32300f2ed9f>:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_['FloorPlan'] = df_.loc[:,['FloorPlan']].applymap(to_full_width)


,Type
FloorPlan,
5LDK,18
others,35
1R,147
4LDK,882
1LDK,1035
1K,2785
2LDK,2869
3LDK,6775


In [68]:
result_df.FloorPlan.isna().sum()

15002

In [60]:

result_df[['FloorPlan', 'FloorPlanOrigin']].drop_duplicates(keep='first')


<ipython-input-53-5c1730ecbfc3>:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_['FloorPlan'] = df_.loc[:,['FloorPlan']].applymap(to_full_width)


,FloorPlan,FloorPlanOrigin
150,3LDK,３ＬＤＫ
153,NaN,NaN
125,4LDK,４ＬＤＫ
137,2LDK,２ＬＤＫ
394,4LDK,３ＬＤＫ＋Ｓ
347,1LDK,１ＬＤＫ
340,1K,１Ｋ
447,3LDK,２ＬＤＫ＋Ｓ
957,1LDK,２ＤＫ
427,2LDK,３ＤＫ


,Type
FloorPlan,
5LDK,18
others,35
1R,147
4LDK,882
1LDK,1035
1K,2785
2LDK,2869
3LDK,6775


In [62]:
result_df.FloorPlan.unique()

array(['3LDK', nan, '4LDK', '2LDK', '1LDK', '1K', '1R', 'others', '5LDK'],
      dtype=object)